In [5]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv(r'C:\Users\ysyzy\.ipynb_checkpoints\Customer_analysis\customer_shopping_behavior.csv')

In [7]:
df.head(2)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [9]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [10]:
df.shape

(3900, 18)

In [11]:
df.isnull().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [12]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [13]:
df.columns = df.columns.str.lower().str.replace(' ','_')

In [14]:
df = df.rename(columns={'purchase_amount_(usd)':'purchase_amount'})

In [15]:
labels = ['young adult','adult','mid-age','senior']
df['age_group'] = pd.qcut(df['age'],q=4,labels=labels)

In [16]:
df[['age_group','age']].head()

,age_group,age
0,mid-age,55
1,young adult,19
2,mid-age,50
3,young adult,21
4,mid-age,45


In [17]:
mapping = {
    'Fortnightly':14,
    'Weekly':7,
    'Annually':365,
    'Quarterly':90,
    'Bi-Weekly':14,
    'Every 3 Months':90,
    'Monthly':30
}
df['frequency_of_purchases_days'] = df['frequency_of_purchases'].map(mapping)

In [18]:
df[['frequency_of_purchases_days','frequency_of_purchases']].head()

,frequency_of_purchases_days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually


In [19]:
(df['promo_code_used'] == df['discount_applied']).all

<bound method Series.all of 0       True
1       True
2       True
3       True
4       True
        ... 
3895    True
3896    True
3897    True
3898    True
3899    True
Length: 3900, dtype: bool>

In [20]:
df = df.drop('promo_code_used',axis=1)

In [21]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Step 1: Connect to PostgreSQL
# Replace placeholders with your actual details
username = "postgres"  # default user
password = "192005Zeyad@" # the password you set during installation
host = "localhost"     # if running locally
port = "5432"          # default PostgreSQL port
database = "customer_behaviour" # the database you created in pgAdmin

password_quoted = quote_plus(password)
engine = create_engine(f"postgresql+psycopg2://{username}:{password_quoted}@{host}:{port}/{database}")

# Step 2: Load DataFrame into PostgreSQL
table_name = "customer"  # choose any table name
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")


Data successfully loaded into table 'customer' in database 'customer_behaviour'.


In [22]:
pip install psycopg2-binary sqlalchemy

Note: you may need to restart the kernel to use updated packages.
